# 🚗 CarV1 Self-Driving Car Training on Google Colab

This notebook trains your reinforcement learning agent for the CarV1 line-following car using **GPU acceleration** on Google Colab.

## 📋 What This Notebook Does:
1. ✅ Checks for GPU availability
2. ✅ Installs required packages
3. ✅ Uploads your project files
4. ✅ Trains the model with PPO algorithm
5. ✅ Monitors training with TensorBoard
6. ✅ Downloads trained models

## ⚡ Estimated Training Time:
- **With GPU (T4)**: 5-10 minutes for 1M timesteps
- **Without GPU (CPU)**: 30-60 minutes

## 🎯 Before You Start:
Make sure you have your project files ready to upload, or a GitHub repo URL.

## Step 1: Check GPU and Environment

First, let's verify we're on Colab and check if GPU is available.

In [ ]:
import sys
import os

# Check if running on Colab
try:
    import google.colab
    IN_COLAB = True
    print("✓ Running on Google Colab")
except ImportError:
    IN_COLAB = False
    print("✗ Not running on Google Colab")

# Check GPU availability
import torch
if torch.cuda.is_available():
    print(f"✓ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"  GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠ No GPU detected - training will be slower")
    print("  To enable GPU: Runtime → Change runtime type → Hardware accelerator → GPU")

print(f"\nPython Version: {sys.version.split()[0]}")
print(f"PyTorch Version: {torch.__version__}")

## Step 2: Install Dependencies

Install all required packages for training.

In [ ]:
%%capture
# Install packages (output suppressed for cleanliness)
!pip install gymnasium pygame opencv-python stable-baselines3[extra] tensorboard

print("✓ All packages installed successfully!")

## Step 3: Upload Project Files

Choose one of the following options to get your code into Colab.

### Option A: Clone from GitHub (Recommended)

If you've pushed your code to GitHub, this is the easiest option.

In [ ]:
# Option A: Clone from GitHub
REPO_URL = "https://github.com/egonthemighty/CarV1.git"

# Clone and navigate to project
!git clone {REPO_URL}
%cd CarV1

print("✓ Repository cloned successfully!")

### Option B: Upload ZIP File (SKIP THIS if using GitHub)

**⚠️ Only use this if you're NOT using GitHub Option A above.**

If not using GitHub, upload your project as a ZIP file.

In [ ]:
# ⚠️ SKIP THIS CELL if you used GitHub Option A above!

# Option B: Upload ZIP file
# First, create a ZIP on your local machine:
#   Run: create_colab_package.bat
#   This creates CarV1.zip with all necessary files

if IN_COLAB:
    from google.colab import files
    print("Click 'Choose Files' and upload your CarV1.zip")
    uploaded = files.upload()
    
    # Extract the ZIP
    import zipfile
    for filename in uploaded.keys():
        if filename.endswith('.zip'):
            print(f"Extracting {filename}...")
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall('.')
            print("✓ Files extracted")
            
            # Try to find the project directory
            if os.path.exists('CarV1'):
                os.chdir('CarV1')
                print("✓ Changed to CarV1 directory")
            break
else:
    print("Skipping upload - not on Colab")

## Step 4: Verify Project Structure

Check that all required files are present.

In [ ]:
# Check project structure
# First ensure we're in the right directory
import os
if os.path.exists('CarV1') and os.getcwd() == '/content':
    %cd CarV1
    print("✓ Changed to CarV1 directory\n")

required_files = [
    'env/camera_line_follow_env.py',
    'tests/train_gradient.py',
    'config/config.py',
]

print("Checking project structure...")
print(f"Current directory: {os.getcwd()}\n")

all_found = True
for file in required_files:
    exists = os.path.exists(file)
    status = "✓" if exists else "✗"
    print(f"{status} {file}")
    if not exists:
        all_found = False

if all_found:
    print("\n✓ All required files found!")
else:
    print("\n⚠ Some files are missing. Check your upload.")
    print("Expected structure:")
    print("  CarV1/")
    print("  ├── env/")
    print("  ├── tests/")
    print("  ├── config/")
    print("  ├── utils/")
    print("  └── references/")

## Step 5: Start Training! 🚀

Now let's train the model. This uses the GPU-optimized training script.

In [ ]:
# Run training
!python tests/train_gradient.py

## Step 6: Monitor Training with TensorBoard (Optional)

You can visualize training progress in real-time.

In [ ]:
# Load TensorBoard extension
%load_ext tensorboard

# Start TensorBoard (run this in a separate cell while training)
%tensorboard --logdir logs

## Step 7: Download Trained Models

After training completes, download your models to your local machine.

In [ ]:
# Create a ZIP of all models and download
import shutil

print("Creating models archive...")
shutil.make_archive('carv1_models', 'zip', 'models')

if IN_COLAB:
    from google.colab import files
    print("Downloading models...")
    files.download('carv1_models.zip')
    print("✓ Download started!")
else:
    print("✓ Models saved to: carv1_models.zip")

print("\nYour trained models include:")
print("  - best_model/best_model.zip - Best performing checkpoint")
print("  - ppo_camera_line_follow_final.zip - Final model after all training")
print("  - checkpoints/ - Intermediate checkpoints every 10k steps")

## 🎉 Training Complete!

### Next Steps:

1. **Test Locally**: Extract the models ZIP and test with your visualization scripts
2. **Adjust Training**: If needed, modify `config/config.py` and re-run
3. **Deploy to Pi**: Follow the hardware integration guide in `references/`

### Helpful Resources:
- 📖 [Project Overview](references/project_overview.md)
- 🔧 [Hardware Integration](references/hardware_integration.md)  
- 🤖 [Training Guide](references/training_guide.md)

### Training Configuration Used:
- Algorithm: PPO (Proximal Policy Optimization)
- Total Timesteps: 1,000,000
- Device: GPU (CUDA) if available
- Environment: Camera-based line following
- Observation: 4 features [left_offset, right_offset, heading, speed]

### Performance Tips:
- Increase timesteps to 5M or 10M for better results
- Monitor TensorBoard graphs for convergence
- Best model is saved automatically based on evaluation rewards